In [5]:
from snowflake_llm.snowflake_utils import get_connection_url,get_lagchain_connection

In [8]:
db=get_lagchain_connection()

In [12]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many locations are there"})
response

'SELECT COUNT(location_id) AS num_locations\nFROM locations;'

In [14]:
db.run(response)

'[(5,)]'

In [15]:
chain.get_prompts()[0].pretty_print()


Given an input question, first create a syntactically correct snowflake query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most 5 results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}


In [25]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
# chain.invoke({"question": "How many locations are there"})
# chain.invoke({"question": "what are the locations for order id 1"})


In [27]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [28]:
res=chain.invoke({"question": "describe orders with prduct A including locations"})

In [29]:
res

"The SQL query retrieves information about orders that include 'Product A' and also includes the locations where these orders were placed. The result shows the order ID, product name, quantity, order date, order amount, location name, city, and state for each order that meets the criteria.\n\nFor example, the result includes orders for 'Product A' placed at Warehouse A in New York, Warehouse B in Los Angeles, Warehouse C in Chicago, Warehouse D in Houston, and Warehouse E in Miami. Each order includes details such as quantity, order date, and order amount along with the location information."